# Check GPU availability
Check GPU drivers and CUDA availability for training.

In [ ]:
# Check GPU status
!nvidia-smi

# Install required packages
Install required packages for downloading dataset and training YOLOv11 model.

In [ ]:
# Install required packages (without comet_ml)
print("Installing required packages...")
%pip install -q ultralytics roboflow matplotlib seaborn pandas
print("Packages installed successfully!")

In [ ]:
# Import necessary libraries
import os
from google.colab import userdata

# Roboflow configuration
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
ROBOFLOW_WORKSPACE = userdata.get('ROBOFLOW_WORKSPACE')
ROBOFLOW_PROJECT = userdata.get('ROBOFLOW_PROJECT')
ROBOFLOW_VERSION = userdata.get('ROBOFLOW_VERSION')

print("[OK] Configuration loaded successfully!")

# Download dataset from Roboflow
Download dataset from Roboflow for YOLOv11 training.

In [ ]:
# Download dataset
from roboflow import Roboflow

print("Downloading dataset from Roboflow...")
print(f"Workspace: {ROBOFLOW_WORKSPACE}")
print(f"Project: {ROBOFLOW_PROJECT}")
print(f"Version: {ROBOFLOW_VERSION}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(int(ROBOFLOW_VERSION))
dataset = version.download("yolov11")

print("Dataset downloaded successfully!")

# Model Evaluation - Single Model

Evaluate a single YOLOv11 segmentation model on the test dataset.
This section will:
1. Load the model
2. Run validation on the test set
3. Collect and display all metrics
4. Visualize results with charts

In [ ]:
# Evaluate both PyTorch and OpenVINO models and collect metrics
import os
import pandas as pd
from ultralytics import YOLO

# Define model paths
PYTORCH_MODEL_PATH = "/content/yolo11n-seg-version-1-0-0.pt"
OPENVINO_MODEL_PATH = "/content/yolo11n-seg-version-1-0-0_int8_openvino_model"

# Get data.yaml path from downloaded dataset
data_yaml = f"{dataset.location}/data.yaml"

print("=" * 70)
print("EVALUATING MODELS ON TEST DATASET")
print("=" * 70)

# Dictionary to store metrics for both models
all_metrics = {}

# Evaluate PyTorch model
print("\n[1/2] Evaluating PyTorch Model...")
print("-" * 70)
print(f"Model: {os.path.basename(PYTORCH_MODEL_PATH)}")
print(f"Path: {PYTORCH_MODEL_PATH}")
print("-" * 70)

pytorch_model = YOLO(PYTORCH_MODEL_PATH)
pytorch_metrics = pytorch_model.val(data=data_yaml, split='test', verbose=False)

# Store PyTorch metrics
all_metrics['PyTorch'] = {
    'Box_mAP50-95': pytorch_metrics.box.map,
    'Box_mAP50': pytorch_metrics.box.map50,
    'Box_mAP75': pytorch_metrics.box.map75,
    'Box_Precision': pytorch_metrics.box.mp,
    'Box_Recall': pytorch_metrics.box.mr,
    'Mask_mAP50-95': pytorch_metrics.seg.map,
    'Mask_mAP50': pytorch_metrics.seg.map50,
    'Mask_mAP75': pytorch_metrics.seg.map75,
    'Mask_Precision': pytorch_metrics.seg.mp,
    'Mask_Recall': pytorch_metrics.seg.mr,
}

print("\nPyTorch Model - BOUNDING BOX METRICS:")
print(f"  mAP50-95:  {pytorch_metrics.box.map:.4f}")
print(f"  mAP50:     {pytorch_metrics.box.map50:.4f}")
print(f"  mAP75:     {pytorch_metrics.box.map75:.4f}")
print(f"  Precision: {pytorch_metrics.box.mp:.4f}")
print(f"  Recall:    {pytorch_metrics.box.mr:.4f}")

print("\nPyTorch Model - SEGMENTATION MASK METRICS:")
print(f"  mAP50-95:  {pytorch_metrics.seg.map:.4f}")
print(f"  mAP50:     {pytorch_metrics.seg.map50:.4f}")
print(f"  mAP75:     {pytorch_metrics.seg.map75:.4f}")
print(f"  Precision: {pytorch_metrics.seg.mp:.4f}")
print(f"  Recall:    {pytorch_metrics.seg.mr:.4f}")

# Evaluate OpenVINO model
print("\n" + "=" * 70)
print("[2/2] Evaluating OpenVINO INT8 Model...")
print("-" * 70)
print(f"Model: {os.path.basename(OPENVINO_MODEL_PATH)}")
print(f"Path: {OPENVINO_MODEL_PATH}")
print("-" * 70)

openvino_model = YOLO(OPENVINO_MODEL_PATH)
openvino_metrics = openvino_model.val(data=data_yaml, split='test', verbose=False)

# Store OpenVINO metrics
all_metrics['OpenVINO_INT8'] = {
    'Box_mAP50-95': openvino_metrics.box.map,
    'Box_mAP50': openvino_metrics.box.map50,
    'Box_mAP75': openvino_metrics.box.map75,
    'Box_Precision': openvino_metrics.box.mp,
    'Box_Recall': openvino_metrics.box.mr,
    'Mask_mAP50-95': openvino_metrics.seg.map,
    'Mask_mAP50': openvino_metrics.seg.map50,
    'Mask_mAP75': openvino_metrics.seg.map75,
    'Mask_Precision': openvino_metrics.seg.mp,
    'Mask_Recall': openvino_metrics.seg.mr,
}

print("\nOpenVINO INT8 Model - BOUNDING BOX METRICS:")
print(f"  mAP50-95:  {openvino_metrics.box.map:.4f}")
print(f"  mAP50:     {openvino_metrics.box.map50:.4f}")
print(f"  mAP75:     {openvino_metrics.box.map75:.4f}")
print(f"  Precision: {openvino_metrics.box.mp:.4f}")
print(f"  Recall:    {openvino_metrics.box.mr:.4f}")

print("\nOpenVINO INT8 Model - SEGMENTATION MASK METRICS:")
print(f"  mAP50-95:  {openvino_metrics.seg.map:.4f}")
print(f"  mAP50:     {openvino_metrics.seg.map50:.4f}")
print(f"  mAP75:     {openvino_metrics.seg.map75:.4f}")
print(f"  Precision: {openvino_metrics.seg.mp:.4f}")
print(f"  Recall:    {openvino_metrics.seg.mr:.4f}")

# Create comparison DataFrame
results_df = pd.DataFrame(all_metrics).T

print("\n" + "=" * 70)
print("COMPARISON SUMMARY")
print("=" * 70)
print("\nBox Metrics Comparison:")
print(f"{'Metric':<20} {'PyTorch':<15} {'OpenVINO INT8':<15} {'Difference':<15}")
print("-" * 65)
print(f"{'mAP50-95':<20} {pytorch_metrics.box.map:<15.4f} {openvino_metrics.box.map:<15.4f} {openvino_metrics.box.map - pytorch_metrics.box.map:+.4f}")
print(f"{'mAP50':<20} {pytorch_metrics.box.map50:<15.4f} {openvino_metrics.box.map50:<15.4f} {openvino_metrics.box.map50 - pytorch_metrics.box.map50:+.4f}")
print(f"{'mAP75':<20} {pytorch_metrics.box.map75:<15.4f} {openvino_metrics.box.map75:<15.4f} {openvino_metrics.box.map75 - pytorch_metrics.box.map75:+.4f}")
print(f"{'Precision':<20} {pytorch_metrics.box.mp:<15.4f} {openvino_metrics.box.mp:<15.4f} {openvino_metrics.box.mp - pytorch_metrics.box.mp:+.4f}")
print(f"{'Recall':<20} {pytorch_metrics.box.mr:<15.4f} {openvino_metrics.box.mr:<15.4f} {openvino_metrics.box.mr - pytorch_metrics.box.mr:+.4f}")

print("\nMask Metrics Comparison:")
print(f"{'Metric':<20} {'PyTorch':<15} {'OpenVINO INT8':<15} {'Difference':<15}")
print("-" * 65)
print(f"{'mAP50-95':<20} {pytorch_metrics.seg.map:<15.4f} {openvino_metrics.seg.map:<15.4f} {openvino_metrics.seg.map - pytorch_metrics.seg.map:+.4f}")
print(f"{'mAP50':<20} {pytorch_metrics.seg.map50:<15.4f} {openvino_metrics.seg.map50:<15.4f} {openvino_metrics.seg.map50 - pytorch_metrics.seg.map50:+.4f}")
print(f"{'mAP75':<20} {pytorch_metrics.seg.map75:<15.4f} {openvino_metrics.seg.map75:<15.4f} {openvino_metrics.seg.map75 - pytorch_metrics.seg.map75:+.4f}")
print(f"{'Precision':<20} {pytorch_metrics.seg.mp:<15.4f} {openvino_metrics.seg.mp:<15.4f} {openvino_metrics.seg.mp - pytorch_metrics.seg.mp:+.4f}")
print(f"{'Recall':<20} {pytorch_metrics.seg.mr:<15.4f} {openvino_metrics.seg.mr:<15.4f} {openvino_metrics.seg.mr - pytorch_metrics.seg.mr:+.4f}")

print("\n" + "=" * 70)
print("[OK] Model evaluation completed!")
print("=" * 70)

# Visualization - Box Metrics Comparison

Visualize Box (Bounding Box) metrics comparison between PyTorch and OpenVINO INT8 models using 4 charts:
1. PyTorch model metrics bar chart
2. OpenVINO INT8 model metrics bar chart
3. Side-by-side comparison
4. Radar chart comparison

In [ ]:
# Visualization - Box Metrics Comparison Charts
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Configuration
Y_AXIS_MIN = 0.9
Y_AXIS_MAX = 1.01
RADAR_MIN = 0.9
RADAR_MAX = 1.01

# Prepare data for Box metrics
metric_names = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
pytorch_box_values = [
    pytorch_metrics.box.map,
    pytorch_metrics.box.map50,
    pytorch_metrics.box.map75,
    pytorch_metrics.box.mp,
    pytorch_metrics.box.mr
]
openvino_box_values = [
    openvino_metrics.box.map,
    openvino_metrics.box.map50,
    openvino_metrics.box.map75,
    openvino_metrics.box.mp,
    openvino_metrics.box.mr
]

# Color palettes
pytorch_color = '#3498db'  # Blue
openvino_color = '#e74c3c'  # Red
comparison_colors = ['#5dade2', '#f5b041']  # Soft blue and amber

# Create figure with subplots for Box metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Box Metrics Comparison: PyTorch vs OpenVINO INT8', fontsize=16, fontweight='bold')

# 1. PyTorch Box Metrics Bar Chart
ax1 = axes[0, 0]
bars1 = ax1.bar(metric_names, pytorch_box_values, color=pytorch_color, edgecolor='black', linewidth=0.5)
ax1.set_ylabel('Score', fontsize=11)
ax1.set_title('PyTorch Model - Box Metrics', fontsize=12, fontweight='bold')
ax1.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

for bar, val in zip(bars1, pytorch_box_values):
    height = bar.get_height()
    ax1.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 2. OpenVINO Box Metrics Bar Chart
ax2 = axes[0, 1]
bars2 = ax2.bar(metric_names, openvino_box_values, color=openvino_color, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Score', fontsize=11)
ax2.set_title('OpenVINO INT8 Model - Box Metrics', fontsize=12, fontweight='bold')
ax2.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

for bar, val in zip(bars2, openvino_box_values):
    height = bar.get_height()
    ax2.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 3. Side-by-side Comparison
ax3 = axes[1, 0]
x = np.arange(len(metric_names))
width = 0.35

bars_pytorch = ax3.bar(x - width/2, pytorch_box_values, width, label='PyTorch',
                       color=pytorch_color, edgecolor='black', linewidth=0.5)
bars_openvino = ax3.bar(x + width/2, openvino_box_values, width, label='OpenVINO INT8',
                        color=openvino_color, edgecolor='black', linewidth=0.5)

ax3.set_ylabel('Score', fontsize=11)
ax3.set_title('PyTorch vs OpenVINO INT8 - Box Metrics Comparison', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(metric_names)
ax3.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), framealpha=0.9)
ax3.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

# 4. Radar Chart
ax4 = plt.subplot(2, 2, 4, polar=True)

# Prepare data for radar chart
N = len(metric_names)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

pytorch_vals = pytorch_box_values + [pytorch_box_values[0]]
openvino_vals = openvino_box_values + [openvino_box_values[0]]

# Configure radar chart
ax4.set_theta_offset(np.pi / 2)
ax4.set_theta_direction(-1)
plt.xticks(angles[:-1], metric_names, fontsize=10)
ax4.set_rlabel_position(0)

radar_ticks = np.linspace(RADAR_MIN, RADAR_MAX, 5)
radar_labels = [f"{t:.2f}" for t in radar_ticks]
plt.yticks(radar_ticks, radar_labels, color="grey", size=8)
plt.ylim(RADAR_MIN, RADAR_MAX)

# Plot data
ax4.plot(angles, pytorch_vals, 'o-', linewidth=2, label='PyTorch', color=pytorch_color)
ax4.fill(angles, pytorch_vals, alpha=0.25, color=pytorch_color)
ax4.plot(angles, openvino_vals, 'o-', linewidth=2, label='OpenVINO INT8', color=openvino_color)
ax4.fill(angles, openvino_vals, alpha=0.25, color=openvino_color)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax4.set_title('Box Metrics Radar Comparison', fontsize=12, fontweight='bold', y=1.08)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('box_metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n[OK] Box metrics comparison chart saved to 'box_metrics_comparison.png'")

# Visualization - Mask Metrics Comparison

Visualize Mask (Segmentation) metrics comparison between PyTorch and OpenVINO INT8 models using 4 charts:
1. PyTorch model metrics bar chart
2. OpenVINO INT8 model metrics bar chart
3. Side-by-side comparison
4. Radar chart comparison

In [ ]:
# Visualization - Mask Metrics Comparison Charts
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Configuration
Y_AXIS_MIN = 0.9
Y_AXIS_MAX = 1.01
RADAR_MIN = 0.9
RADAR_MAX = 1.01

# Prepare data for Mask metrics
metric_names = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
pytorch_mask_values = [
    pytorch_metrics.seg.map,
    pytorch_metrics.seg.map50,
    pytorch_metrics.seg.map75,
    pytorch_metrics.seg.mp,
    pytorch_metrics.seg.mr
]
openvino_mask_values = [
    openvino_metrics.seg.map,
    openvino_metrics.seg.map50,
    openvino_metrics.seg.map75,
    openvino_metrics.seg.mp,
    openvino_metrics.seg.mr
]

# Color palettes
pytorch_color = '#2ecc71'  # Green
openvino_color = '#e67e22'  # Orange
comparison_colors = ['#52b788', '#f39c12']  # Soft green and orange

# Create figure with subplots for Mask metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Mask (Segmentation) Metrics Comparison: PyTorch vs OpenVINO INT8', fontsize=16, fontweight='bold')

# 1. PyTorch Mask Metrics Bar Chart
ax1 = axes[0, 0]
bars1 = ax1.bar(metric_names, pytorch_mask_values, color=pytorch_color, edgecolor='black', linewidth=0.5)
ax1.set_ylabel('Score', fontsize=11)
ax1.set_title('PyTorch Model - Mask Metrics', fontsize=12, fontweight='bold')
ax1.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

for bar, val in zip(bars1, pytorch_mask_values):
    height = bar.get_height()
    ax1.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 2. OpenVINO Mask Metrics Bar Chart
ax2 = axes[0, 1]
bars2 = ax2.bar(metric_names, openvino_mask_values, color=openvino_color, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Score', fontsize=11)
ax2.set_title('OpenVINO INT8 Model - Mask Metrics', fontsize=12, fontweight='bold')
ax2.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

for bar, val in zip(bars2, openvino_mask_values):
    height = bar.get_height()
    ax2.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 3. Side-by-side Comparison
ax3 = axes[1, 0]
x = np.arange(len(metric_names))
width = 0.35

bars_pytorch = ax3.bar(x - width/2, pytorch_mask_values, width, label='PyTorch',
                       color=pytorch_color, edgecolor='black', linewidth=0.5)
bars_openvino = ax3.bar(x + width/2, openvino_mask_values, width, label='OpenVINO INT8',
                        color=openvino_color, edgecolor='black', linewidth=0.5)

ax3.set_ylabel('Score', fontsize=11)
ax3.set_title('PyTorch vs OpenVINO INT8 - Mask Metrics Comparison', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(metric_names)
ax3.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), framealpha=0.9)
ax3.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

# 4. Radar Chart
ax4 = plt.subplot(2, 2, 4, polar=True)

# Prepare data for radar chart
N = len(metric_names)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

pytorch_vals = pytorch_mask_values + [pytorch_mask_values[0]]
openvino_vals = openvino_mask_values + [openvino_mask_values[0]]

# Configure radar chart
ax4.set_theta_offset(np.pi / 2)
ax4.set_theta_direction(-1)
plt.xticks(angles[:-1], metric_names, fontsize=10)
ax4.set_rlabel_position(0)

radar_ticks = np.linspace(RADAR_MIN, RADAR_MAX, 5)
radar_labels = [f"{t:.2f}" for t in radar_ticks]
plt.yticks(radar_ticks, radar_labels, color="grey", size=8)
plt.ylim(RADAR_MIN, RADAR_MAX)

# Plot data
ax4.plot(angles, pytorch_vals, 'o-', linewidth=2, label='PyTorch', color=pytorch_color)
ax4.fill(angles, pytorch_vals, alpha=0.25, color=pytorch_color)
ax4.plot(angles, openvino_vals, 'o-', linewidth=2, label='OpenVINO INT8', color=openvino_color)
ax4.fill(angles, openvino_vals, alpha=0.25, color=openvino_color)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax4.set_title('Mask Metrics Radar Comparison', fontsize=12, fontweight='bold', y=1.08)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('mask_metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n[OK] Mask metrics comparison chart saved to 'mask_metrics_comparison.png'")

# Save Results to Google Drive

Save evaluation results to Google Drive:
1. Text file with detailed metrics comparison
2. Box metrics comparison chart
3. Mask metrics comparison chart

In [ ]:
# Save results to Google Drive
import os
import shutil
from datetime import datetime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create output directory
output_dir = "/content/drive/MyDrive/Metrics"
os.makedirs(output_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print("=" * 70)
print("SAVING RESULTS TO GOOGLE DRIVE")
print("=" * 70)
print(f"Output directory: {output_dir}")
print("-" * 70)

# 1. Save metrics as text file
metrics_file = os.path.join(output_dir, f"evaluation_metrics_{timestamp}.txt")
with open(metrics_file, 'w') as f:
    f.write("=" * 70 + "\n")
    f.write("YOLO11n-seg Model Comparison: PyTorch vs OpenVINO INT8\n")
    f.write(f"Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 70 + "\n\n")

    f.write("MODEL PATHS:\n")
    f.write("-" * 40 + "\n")
    f.write(f"PyTorch Model:  {PYTORCH_MODEL_PATH}\n")
    f.write(f"OpenVINO Model: {OPENVINO_MODEL_PATH}\n\n")

    f.write("BOUNDING BOX METRICS COMPARISON:\n")
    f.write("=" * 70 + "\n")
    f.write(f"{'Metric':<20} {'PyTorch':<15} {'OpenVINO INT8':<15} {'Difference':<15}\n")
    f.write("-" * 70 + "\n")
    f.write(f"{'mAP50-95':<20} {pytorch_metrics.box.map:<15.4f} {openvino_metrics.box.map:<15.4f} {openvino_metrics.box.map - pytorch_metrics.box.map:+.4f}\n")
    f.write(f"{'mAP50':<20} {pytorch_metrics.box.map50:<15.4f} {openvino_metrics.box.map50:<15.4f} {openvino_metrics.box.map50 - pytorch_metrics.box.map50:+.4f}\n")
    f.write(f"{'mAP75':<20} {pytorch_metrics.box.map75:<15.4f} {openvino_metrics.box.map75:<15.4f} {openvino_metrics.box.map75 - pytorch_metrics.box.map75:+.4f}\n")
    f.write(f"{'Precision':<20} {pytorch_metrics.box.mp:<15.4f} {openvino_metrics.box.mp:<15.4f} {openvino_metrics.box.mp - pytorch_metrics.box.mp:+.4f}\n")
    f.write(f"{'Recall':<20} {pytorch_metrics.box.mr:<15.4f} {openvino_metrics.box.mr:<15.4f} {openvino_metrics.box.mr - pytorch_metrics.box.mr:+.4f}\n\n")

    f.write("SEGMENTATION MASK METRICS COMPARISON:\n")
    f.write("=" * 70 + "\n")
    f.write(f"{'Metric':<20} {'PyTorch':<15} {'OpenVINO INT8':<15} {'Difference':<15}\n")
    f.write("-" * 70 + "\n")
    f.write(f"{'mAP50-95':<20} {pytorch_metrics.seg.map:<15.4f} {openvino_metrics.seg.map:<15.4f} {openvino_metrics.seg.map - pytorch_metrics.seg.map:+.4f}\n")
    f.write(f"{'mAP50':<20} {pytorch_metrics.seg.map50:<15.4f} {openvino_metrics.seg.map50:<15.4f} {openvino_metrics.seg.map50 - pytorch_metrics.seg.map50:+.4f}\n")
    f.write(f"{'mAP75':<20} {pytorch_metrics.seg.map75:<15.4f} {openvino_metrics.seg.map75:<15.4f} {openvino_metrics.seg.map75 - pytorch_metrics.seg.map75:+.4f}\n")
    f.write(f"{'Precision':<20} {pytorch_metrics.seg.mp:<15.4f} {openvino_metrics.seg.mp:<15.4f} {openvino_metrics.seg.mp - pytorch_metrics.seg.mp:+.4f}\n")
    f.write(f"{'Recall':<20} {pytorch_metrics.seg.mr:<15.4f} {openvino_metrics.seg.mr:<15.4f} {openvino_metrics.seg.mr - pytorch_metrics.seg.mr:+.4f}\n\n")

    # Calculate average degradation
    box_degradation = (openvino_metrics.box.map - pytorch_metrics.box.map) / pytorch_metrics.box.map * 100
    mask_degradation = (openvino_metrics.seg.map - pytorch_metrics.seg.map) / pytorch_metrics.seg.map * 100

    f.write("PERFORMANCE SUMMARY:\n")
    f.write("-" * 40 + "\n")
    f.write(f"Box mAP50-95 Degradation:  {box_degradation:+.2f}%\n")
    f.write(f"Mask mAP50-95 Degradation: {mask_degradation:+.2f}%\n\n")

    if abs(box_degradation) < 2.0 and abs(mask_degradation) < 2.0:
        f.write("ANALYSIS: INT8 quantization has minimal impact on model performance.\n")
        f.write("The OpenVINO INT8 model maintains comparable accuracy with reduced size.\n")
    elif abs(box_degradation) < 5.0 and abs(mask_degradation) < 5.0:
        f.write("ANALYSIS: INT8 quantization shows acceptable performance trade-off.\n")
        f.write("The accuracy degradation is within acceptable range for deployment.\n")
    else:
        f.write("ANALYSIS: INT8 quantization shows significant performance degradation.\n")
        f.write("Consider using FP16 quantization or additional calibration.\n")

print(f"\n[1/3] Metrics saved to: {metrics_file}")

# 2. Copy Box metrics chart
box_chart_src = "box_metrics_comparison.png"
box_chart_dest = os.path.join(output_dir, f"box_metrics_comparison_{timestamp}.png")
if os.path.exists(box_chart_src):
    shutil.copy2(box_chart_src, box_chart_dest)
    print(f"[2/3] Box metrics chart saved to: {box_chart_dest}")
else:
    print(f"[2/3] Warning: Box metrics chart not found at {box_chart_src}")

# 3. Copy Mask metrics chart
mask_chart_src = "mask_metrics_comparison.png"
mask_chart_dest = os.path.join(output_dir, f"mask_metrics_comparison_{timestamp}.png")
if os.path.exists(mask_chart_src):
    shutil.copy2(mask_chart_src, mask_chart_dest)
    print(f"[3/3] Mask metrics chart saved to: {mask_chart_dest}")
else:
    print(f"[3/3] Warning: Mask metrics chart not found at {mask_chart_src}")

# Summary
print("\n" + "=" * 70)
print("SAVE SUMMARY")
print("=" * 70)
print(f"\nAll results saved to Google Drive:")
print(f"Directory: {output_dir}/")
print(f"  - evaluation_metrics_{timestamp}.txt")
print(f"  - box_metrics_comparison_{timestamp}.png")
print(f"  - mask_metrics_comparison_{timestamp}.png")
print("\n[OK] All results saved successfully to Google Drive!")
print("=" * 70)